# Ch `02`: Concept `09`

## Using Queues

If you have a lot of training data, you probably don't want to load it all into memory at once. The QueueRunner in TensorFlow is a tool to efficiently employ a queue data-structure in a multi-threaded way.

In [1]:
import tensorflow as tf
import numpy as np

We will be running multiple threads, so let's figure out the number of CPUs:

In [2]:
import multiprocessing
NUM_THREADS = multiprocessing.cpu_count()

Generate some fake data to work with:

In [3]:
xs = np.random.randn(100, 3)
ys = np.random.randint(0, 2, size=100)

Here's a couple concrete examples of our data:

In [4]:
xs_and_ys = zip(xs, ys)
for _ in range(5):
    x, y = next(xs_and_ys)
    print('Input {}  --->  Output {}'.format(x, y))

Input [ 0.90767397  0.0632223   0.31398435]  --->  Output 1
Input [ 0.16930178  1.56334734  1.32234303]  --->  Output 0
Input [-1.25690393 -1.77570697  0.76984091]  --->  Output 1
Input [-0.46448889  1.76256201 -0.96192379]  --->  Output 0
Input [-0.20284244  1.64780812 -1.72736262]  --->  Output 1


Define a queue:

In [5]:
queue = tf.FIFOQueue(capacity=1000, dtypes=[tf.float32, tf.int32])

Set up the enqueue and dequeue ops:

In [6]:
enqueue_op = queue.enqueue_many([xs, ys])
x_op, y_op = queue.dequeue()

Define a QueueRunner:

In [7]:
qr = tf.train.QueueRunner(queue, [enqueue_op] * 4)

Now that all variables and ops have been defined, let's get started with a session:

In [8]:
sess = tf.InteractiveSession()

Create threads for the QueueRunner:

In [9]:
coord = tf.train.Coordinator()
enqueue_threads = qr.create_threads(sess, coord=coord, start=True)

Test out dequeueing:

In [10]:
for _ in range(100):
    if coord.should_stop():
        break
    x, y = sess.run([x_op, y_op])
    print(x, y)
coord.request_stop()
coord.join(enqueue_threads)

[ 0.90767395  0.0632223   0.31398433] 1
[ 0.16930178  1.56334734  1.32234299] 0
[-1.25690389 -1.77570701  0.7698409 ] 1
[-0.46448889  1.76256204 -0.96192378] 0
[-0.20284244  1.64780807 -1.72736263] 1
[-0.12239845  0.13719925 -0.62309372] 1
[-0.14866383  0.7648862  -0.19160978] 0
[-1.22472143 -1.52581275  0.17922515] 0
[-0.29043567 -0.56472284  0.44870189] 0
[ 1.08143079  1.82799423  0.99836028] 0
[ 0.20733543 -0.19474593 -0.31554851] 1
[-0.63049948 -0.07175544 -0.5827803 ] 0
[-0.79219991 -0.89796716  1.9700253 ] 1
[-1.74615514 -0.62891144 -0.27554378] 1
[-1.42315328 -1.4939872   0.04312736] 0
[-0.07451641 -0.09488028 -0.60759407] 0
[-0.35007033 -1.97407985 -0.25313482] 1
[ 1.28755605 -1.49392152 -0.21323641] 1
[-0.34860429  0.92558557  0.1552223 ] 1
[-1.44849753 -1.60370064 -0.56467789] 1
[ 0.59765565  1.08645952 -1.31306887] 0
[ 0.74644458  1.90284467  1.40989971] 0
[ 0.29432124  0.68280339 -1.13285708] 0
[-0.44719821 -0.31519744  0.10421325] 0
[ 1.66894114 -0.8937484  -0.21351063] 1


In [ ]:
Test complete; Gopal